# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# Checking the current working directory
print(os.getcwd())

# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # join the file path andn roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    

/Users/hoonna/OneDrive/OneDrive - Georgia Institute of Technology/Georiga_Tech/Spring 2020/Data_Engineering/Project2


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = []

# for each filepath in the file path list
for f in file_path_list:
    
    # reading csv feil
    with open(f, 'r', encoding ='utf8',newline='') as csvfile:
        #creating a csv reader object
        csvreader=csv.reader(csvfile)
        next(csvreader)
        
        #extracting each data row one by one and append it
        for line in csvreader:
            full_data_rows_list.append(line)
print("total number of rows",len(full_data_rows_list))
#print("data",full_data_rows_list)

#creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the\
#Apache Cassandra tables
csv.register_dialect('myDialect',quoting=csv.QUOTE_ALL, skipinitialspace = True)

with open('event_datafile_new.csv','w',encoding ='utf8',newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                    'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if(row[0]==''):
            continue
        writer.writerow((row[0],row[2],row[3],row[4],row[5],row[6],row[7], row[8], row[12], row[13], row[16]))

total number of rows 8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv','r',encoding='utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# Making a connection to a Cassandra instance
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# a session is needed to establish connection and begin executing queries
session = cluster.connect()

#### Create Keyspace

In [6]:
# To create a kespace
session.execute("""CREATE KEYSPACE IF NOT EXISTS project2
WITH REPLICATION =
{'class':'SimpleStrategy','replication_factor':1}""")

#### Set Keyspace

In [7]:
# To set KEYSPACE to the keyspace created (project2)
session.set_keyspace('project2')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
session.execute('DROP TABLE IF EXISTS music_app_history1')

In [11]:
## Query 1:  Give the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
"""
For this table, the composite key of session id and item in session was used as a primary key. Session id is the  
Since these two keys uniquely define each row of the data and there is no sorting necessary, i did not add any more clustering column.
For the columns, along with session id and item in session, i used artist name, song title nad song duration which are needed to output the result.
I belive that session id and item in session are usually in numbers not text, i set the data type as int

"""

query = "CREATE TABLE IF NOT EXISTS music_app_history1"
query = query + "(sessionId int, itemInSession int, artist_name text, song_title text, song_length text, PRIMARY KEY (sessionId, itemInSession))"

session.execute(query)


In [12]:
"""
Insert the corresponding information from our csv file. 
Since the original csv file have session id and item in session as strings, i changed them to int"""
# store the file name
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    # to skip the header
    next(csvreader)
    for line in csvreader:
        # Insert statement
        query = "INSERT INTO music_app_history1 (sessionId, itemInSession, artist_name, song_title, song_length)"
        query = query + "VALUES (%s,%s,%s,%s,%s)"
        
        # Match the data and the table accordingly
        session.execute(query, (int(line[8]),int(line[3]),line[0],line[9],line[5]))

#### Do a SELECT to verify that the data have been inserted into each table

In [13]:
# Validation of the table

query = "SELECT * FROM music_app_history1 "
query = query + "WHERE sessionId = 338 AND itemInSession = 4"
rows = session.execute(query)
for row in rows:
    print(row.artist_name, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.3073


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [14]:
## Query 2: Give only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
"""
For this table, the composite key of user id, session id and item in session are used. Item in session is used because the output needs to be sorted by iteminsesison.
Since the composite key uniquely defines each row of the data, i did not additional clustering columns.
"""
#drop if exists
session.execute("DROP TABLE IF EXISTS music_app_history2")

query = "CREATE TABLE IF NOT EXISTS music_app_history2 "
query = query + "(userId int, sessionId int, itemInSession int, artist_name text, song_title text, first_name text, last_name text, PRIMARY KEY (userId, sessionId, itemInSession))"
session.execute(query)


In [15]:
"""
Insert the corresponding information from our csv file. 
Since the original csv file have session id,item in session, and user id as strings, i changed them to int"""
# store the file name
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        # Insert statement
        query = "INSERT INTO music_app_history2 (userId, sessionId, itemInSession, artist_name, song_title, first_name, last_name) "
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s)"
        
        # to match the data and the table accordingly
        session.execute(query, (int(line[10]),int(line[8]),int(line[3]),line[0],line[9],line[1], line[4]))
        

In [16]:
# Validation step
query = "SELECT * FROM music_app_history2 "
query = query + "WHERE userId = 10 AND sessionId = 182"
rows = session.execute(query)
for row in rows:
    print(row.artist_name, row.song_title, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [17]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
"""
For the table, i used the composite key of song_title, session id and item in session as the primary key.
The primary key uniquely defines each row of the data.
There is no sorting needed.
"""

session.execute("DROP TABLE IF EXISTS music_app_history3")
query = "CREATE TABLE IF NOT EXISTS music_app_history3"
query = query + " (song_title text, sessionId int, itemInSession int, f_name text, l_name text, PRIMARY KEY (song_title, sessionId, itemInSession))"

session.execute(query)               

In [18]:
"""
Insert the corresponding information from our csv file. 
Since the original csv file have session id and item in session as strings, i changed them to int"""
# store the file name
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO music_app_history3 (song_title, sessionId, itemInSession, f_name, l_name)"
        query = query + " VALUES (%s,%s,%s,%s,%s)"
        
        session.execute(query, (line[9],int(line[8]),int(line[3]),line[1],line[4]))

In [19]:
# validation
query = "SELECT * FROM music_app_history3 "
query = query + "WHERE song_title = 'All Hands Against His Own'"

rows = session.execute(query)
for row in rows:
    print(row.f_name, row.l_name)

Sara Johnson
Jacqueline Lynch
Tegan Levine


### Drop the tables before closing out the sessions

In [20]:
## TO-DO: Drop the table before closing out the sessions

In [21]:
session.execute("DROP TABLE IF EXISTS music_app_history1")
session.execute("DROP TABLE IF EXISTS music_app_history2")
session.execute("DROP TABLE IF EXISTS music_app_history3")

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()